<a href="https://colab.research.google.com/github/Frederik-Roeckle/HFF_AT_Assignments/blob/master/USTECH100_BacktestingWaveletTradingStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [23]:
!pip install backtesting
!pip install pywavelets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [88]:
import pandas as pd
from backtesting import Backtest, Strategy
import backtesting.lib
import pandas as pd
from fastai.vision.all import *
from fastai.data.all import *
from fastai.learner import load_learner
import pywt

In [77]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/HFF_AT_Assignment_3/ustech100/testdata_ustech100_minute.txt", sep=", ", header=None)
df.rename(columns={0: 'Datetime', 1: 'Midprice'}, inplace=True)
df["Midprice"] = df["Midprice"].str.replace(",", ".")
df["Midprice"] = df["Midprice"].astype("float")
df["Datetime"] = pd.to_datetime(df['Datetime'], format='%d.%m.%Y %H:%M:%S%f')
df.set_index("Datetime", inplace=True)
df["Open"] = df["Midprice"]
df["High"] = df["Midprice"]
df["Low"] = df["Midprice"]
df["Close"] = df["Midprice"]
df["Volume"] = 1000
df.drop(columns=["Midprice"], inplace=True)
df.head()

<ipython-input-77-d72a60d3d176>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/HFF_AT_Assignment_3/ustech100/testdata_ustech100_minute.txt", sep=", ", header=None)


,Open,High,Low,Close,Volume
Datetime,,,,,
2024-07-05 07:01:00,20220.6,20220.6,20220.6,20220.6,1000
2024-07-05 07:02:00,20221.8,20221.8,20221.8,20221.8,1000
2024-07-05 07:03:00,20223.5,20223.5,20223.5,20223.5,1000
2024-07-05 07:04:00,20224.2,20224.2,20224.2,20224.2,1000
2024-07-05 07:05:00,20225.3,20225.3,20225.3,20225.3,1000


In [11]:
learn = load_learner("/content/drive/MyDrive/Colab Notebooks/HFF_AT_Assignment_3/ustech100/squared_model_five_epochs.pkl")

In [82]:
N_TRAIN = 240
class WaveletLearningTradingStratey(Strategy):
  def init(self):
    self.clf = load_learner("/content/drive/MyDrive/Colab Notebooks/HFF_AT_Assignment_3/ustech100/squared_model_five_epochs.pkl")
    self.sampling_rate = 1  # Minutely data
    self.frequency_range = np.linspace(1, 50, 100)  # Define frequency range for CWT
    self.wavelet = "cmor1.5-1.0"  # Complex Morlet wavelet
    self.counter = 0

  def next(self):
    self.counter += 1

    # Start trading after the first 240 bars are available and only ever other hour
    if len(self.data) < N_TRAIN or self.counter < N_TRAIN:
      return
    self.counter = 0
    df = self.data.df.iloc[-N_TRAIN:]
    time_series = (df.reset_index()).Close
    time_series_normalized = (time_series - time_series.mean()) / time_series.std()

    # Perform Continuous Wavelet Transform (CWT)
    coefficients, frequencies = pywt.cwt(time_series_normalized, self.frequency_range, self.wavelet, self.sampling_rate)

    # Plot the scalogram
    plt.figure(figsize=(4, 4))
    plt.imshow(np.abs(coefficients), extent=[0, len(time_series), self.frequency_range.min(), self.frequency_range.max()],
              aspect='auto', interpolation='bilinear', cmap='jet', vmin=0, vmax=np.percentile(np.abs(coefficients), 95))
    plt.savefig("./temp.png")
    plt.close()

    # Predict the future value with the pre-trained network
    res = learn.predict("./temp.png")
    if res[0] == "up":
      self.buy(size=1)
    elif res[0] == "down":
      self.sell(size=1)

In [ ]:
# Conduct backtesting with backtesting.py
bt = Backtest(df, WaveletLearningTradingStratey, cash=1_000_000, commission=.002, trade_on_close=True)
stats = bt.run()
stats

In [90]:
# Analyse only the long trades since the confidence there was higher in the confusion matrix
only_long_trades = stats._trades[stats._trades.Size > 0]
long_stats = backtesting.lib.compute_stats(stats=stats, trades=only_long_trades, data=df, risk_free_rate=0.0392)
long_stats

/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:92: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  s.loc['Equity Final [$]'] = equity[-1]
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  s.loc['Return [%]'] = (equity[-1] - equity[0]) / equity[0] * 100


,0
Start,2024-07-05 07:01:00
End,2024-08-15 19:59:00
Duration,41 days 12:58:00
Exposure Time [%],99.043613
Equity Final [$],986364.925
Equity Peak [$],1000725.5778
Return [%],-1.363508
Buy & Hold Return [%],-3.7625
Return (Ann.) [%],-11.279696
Volatility (Ann.) [%],2.945684
